In [1]:
using BenchmarkTools

In [2]:
function maxpy!(a::Float64, x::Array{Float64,1}, y::Array{Float64,1})
    n = size(y,1)
    for i=1:n
        y[i] += a * x[i]
    end
end

maxpy! (generic function with 1 method)

In [3]:
a=1.2;

In [4]:
@benchmark maxpy!(a,x,y) setup=(x=rand(1000000); y=copy(x))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.886 ms (0.00% GC)
  median time:      2.026 ms (0.00% GC)
  mean time:        2.029 ms (0.00% GC)
  maximum time:     2.737 ms (0.00% GC)
  --------------
  samples:          707
  evals/sample:     1

In [5]:
@benchmark y+=a*x setup=(x=rand(1000000); y=copy(x))

BenchmarkTools.Trial: 
  memory estimate:  15.26 MiB
  allocs estimate:  4
  --------------
  minimum time:     4.827 ms (0.00% GC)
  median time:      5.482 ms (7.15% GC)
  mean time:        5.518 ms (4.88% GC)
  maximum time:     6.800 ms (6.34% GC)
  --------------
  samples:          447
  evals/sample:     1

In [6]:
function blas_axpy!(a::Float64, x::Array{Float64,1}, y::Array{Float64,1})
    ccall( (:cblas_daxpy, "libopenblas"),
        Void, (Clonglong, Cdouble, Ptr{Cdouble}, Clonglong, Ptr{Cdouble}, Clonglong),
        size(y,1), a, x, 1, y, 1)
end    

blas_axpy! (generic function with 1 method)

In [7]:
@benchmark blas_axpy!(a,x,y) setup=(x=rand(1000000); y=copy(x))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.014 ms (0.00% GC)
  median time:      2.088 ms (0.00% GC)
  mean time:        2.094 ms (0.00% GC)
  maximum time:     2.448 ms (0.00% GC)
  --------------
  samples:          749
  evals/sample:     1

---

In [8]:
function measure_maxpy(n)
    t = @benchmark maxpy!(a,x,y) setup=(x=rand($n); y=copy(x))
    mflops = 2*n / median(t).time * 1.e3
end

measure_maxpy (generic function with 1 method)

In [9]:
measure_maxpy(1000)

2681.6898971658434

In [10]:
sizes = round.(Int, logspace(1,7,15))

15-element Array{Int64,1}:
       10
       27
       72
      193
      518
     1389
     3728
    10000
    26827
    71969
   193070
   517947
  1389495
  3727594
 10000000

In [11]:
mflops = [measure_maxpy(n) for n in sizes]

15-element Array{Float64,1}:
  764.038
 1347.55 
 1984.0  
 2296.71 
 2495.56 
 2596.02 
 2665.29 
 2569.66 
 2595.24 
 2466.97 
 1892.07 
  995.445
  965.5  
  939.592
  916.732

In [ ]:
using PyPlot

In [ ]:
semilogx(sizes, mflops)
grid(true)
xlabel("size"); ylabel("MFLOPS");